In [2]:
from datetime import datetime, timedelta
import re

In [ ]:
#주간/주말 박스오피스 데이터

In [3]:
time = datetime(2020,5,31)
p1 = re.compile('[0-9]{4}[-][0-9]{2}[-][0-9]{2}')

dates=[]
for i in range(50):
    time_new = time +  timedelta(days=-7*i)
    date = p1.findall(str(time_new))
    date = date[0].replace("-","")
    dates.append(date)
print(dates)

['20200531', '20200524', '20200517', '20200510', '20200503', '20200426', '20200419', '20200412', '20200405', '20200329', '20200322', '20200315', '20200308', '20200301', '20200223', '20200216', '20200209', '20200202', '20200126', '20200119', '20200112', '20200105', '20191229', '20191222', '20191215', '20191208', '20191201', '20191124', '20191117', '20191110', '20191103', '20191027', '20191020', '20191013', '20191006', '20190929', '20190922', '20190915', '20190908', '20190901', '20190825', '20190818', '20190811', '20190804', '20190728', '20190721', '20190714', '20190707', '20190630', '20190623']


In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

In [130]:
df = pd.DataFrame(columns = ["movieCd","movieNm","audiAcc"]).set_index("movieCd", drop=True)
df['audiAcc']=0
df

,movieNm,audiAcc
movieCd,,


In [132]:
for date in dates:
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=a04a966fc96003dba10a93a92397a7a1&targetDt="+date+"&weekGb=0"

    req = requests.get(url)
    html = req.json()
    weekly_top10 = html['boxOfficeResult']['weeklyBoxOfficeList']  
    
    for i in range(len(weekly_top10)):
        df.loc[html['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieCd'],'movieCd'] = html['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieCd']
        df.loc[html['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieCd'],'movieNm'] = html['boxOfficeResult']['weeklyBoxOfficeList'][i]['movieNm']
        


In [135]:
df['audiAcc']=0

for date in dates:
    url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=a04a966fc96003dba10a93a92397a7a1&targetDt="+date+"&weekGb=0"

    req = requests.get(url)
    html = req.json()
    weekly_top10 = html['boxOfficeResult']['weeklyBoxOfficeList']  

    for i in range(len(weekly_top10)):
      
        if int(html['boxOfficeResult']['weeklyBoxOfficeList'][i].get("audiAcc")) > int(df.loc[html['boxOfficeResult']['weeklyBoxOfficeList'][i].get("movieCd"),"audiAcc"]):
            df.loc[html['boxOfficeResult']['weeklyBoxOfficeList'][i].get("movieCd"),"audiAcc"] = html['boxOfficeResult']['weeklyBoxOfficeList'][i].get("audiAcc")
        else:
            pass


In [136]:
df

,movieNm,audiAcc,movieCd
movieCd,,,
20179462,위대한 쇼맨,1514467,20179462
20209161,언더워터,58253,20209161
20207893,프리즌 이스케이프,171325,20207893
20208122,카페 벨에포크,42007,20208122
20207501,그집,19455,20207501
...,...,...,...
20192022,마담 싸이코,37437,20192022
20196657,맨 인 블랙: 인터내셔널,843703,20196657
20190885,사탄의 인형,46546,20190885


In [143]:
weekly_top10_df = df.reset_index(drop=True)
weekly_top10_df

,movieNm,audiAcc,movieCd
0,위대한 쇼맨,1514467,20179462
1,언더워터,58253,20209161
2,프리즌 이스케이프,171325,20207893
3,카페 벨에포크,42007,20208122
4,그집,19455,20207501
...,...,...,...
173,마담 싸이코,37437,20192022
174,맨 인 블랙: 인터내셔널,843703,20196657
175,사탄의 인형,46546,20190885
176,엑스맨: 다크 피닉스,861032,20182387


In [147]:
weekly_top10_df.to_csv("box_office.csv", encoding='utf-8-sig')

## 영화 상세정보

In [165]:
df = weekly_top10_df.copy()[['movieNm',"movieCd"]]
df

,movieNm,movieCd
0,위대한 쇼맨,20179462
1,언더워터,20209161
2,프리즌 이스케이프,20207893
3,카페 벨에포크,20208122
4,그집,20207501
...,...,...
173,마담 싸이코,20192022
174,맨 인 블랙: 인터내셔널,20196657
175,사탄의 인형,20190885
176,엑스맨: 다크 피닉스,20182387


In [174]:
df['movieNmEn'] = None
df['movieNmOg'] = None
df['watchGradeNm'] = None
df['prdtYear'] = None
df['showTm'] = None
df['genres'] = None
df['directors'] = None
df.set_index('movieCd', inplace=True)
df

,movieNm,movieNmEn,movieNmOg,watchGradeNm,prdtYear,showTm,genres,directors
movieCd,,,,,,,,
20179462,위대한 쇼맨,None,None,None,None,None,None,None
20209161,언더워터,None,None,None,None,None,None,None
20207893,프리즌 이스케이프,None,None,None,None,None,None,None
20208122,카페 벨에포크,None,None,None,None,None,None,None
20207501,그집,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
20192022,마담 싸이코,None,None,None,None,None,None,None
20196657,맨 인 블랙: 인터내셔널,None,None,None,None,None,None,None
20190885,사탄의 인형,None,None,None,None,None,None,None


In [169]:
movie_code = list(df['movieCd'])

In [225]:
for movie in movie_code:
    url2 = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a04a966fc96003dba10a93a92397a7a1&movieCd="+movie

    req = requests.get(url2)
    html = req.json()    
    
    df.loc[movie,'movieNmEn'] = html['movieInfoResult']['movieInfo']['movieNmEn'] #영화명(영문)
    df.loc[movie, 'movieNmOg'] = html['movieInfoResult']['movieInfo']['movieNmOg'] #영화명(원문)
    df.loc[movie, "prdtYear"] = html['movieInfoResult']['movieInfo']['prdtYear'] #개봉연도
    df.loc[movie, "showTm"] = html['movieInfoResult']['movieInfo']['showTm'] #상영시간
    
    #관람등급
    if len(html['movieInfoResult']['movieInfo']['audits']) == 0:
        pass
    else:
        df.loc[movie, "watchGradeNm"] = html['movieInfoResult']['movieInfo']['audits'][0]['watchGradeNm'] 
    
    #감독
    if len(html['movieInfoResult']['movieInfo']['directors']) == 0:
        pass
    else:
        df.loc[movie, 'directors'] = html['movieInfoResult']['movieInfo']['directors'][0]['peopleNm']

In [226]:
df

,movieNm,movieNmEn,movieNmOg,watchGradeNm,prdtYear,showTm,genres,directors
movieCd,,,,,,,,
20179462,위대한 쇼맨,The Greatest Showman,,12세이상관람가,2017,104,"[드라마, 뮤지컬]",마이클 그레이시
20209161,언더워터,Underwater,,15세이상관람가,2019,94,[스릴러],윌리엄 유뱅크
20207893,프리즌 이스케이프,ESCAPE FROM PRETORIA,,12세이상관람가,2020,106,"[어드벤처, 스릴러]",None
20208122,카페 벨에포크,La belle epoque,,15세이상관람가,2019,115,"[코미디, 드라마]",니콜라스 베도스
20207501,그집,32 Malasana Street,Malasaña 32,15세이상관람가,2020,104,"[공포(호러), 스릴러]",None
...,...,...,...,...,...,...,...,...
20192022,마담 싸이코,Greta,,15세이상관람가,2018,98,"[공포(호러), 스릴러]",닐 조단
20196657,맨 인 블랙: 인터내셔널,Men in Black: International,,12세이상관람가,2019,115,"[액션, 코미디, SF]",F. 게리 그레이
20190885,사탄의 인형,Child's Play,,청소년관람불가,2019,90,[공포(호러)],라스 클리브버그


In [227]:
#장르
for movie in movie_code:
    url2 = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=a04a966fc96003dba10a93a92397a7a1&movieCd="+movie

    req = requests.get(url2)
    html = req.json()  
    
    genres=[]
    for genre in html['movieInfoResult']['movieInfo']['genres']:
        genres.append(genre['genreNm'])
    
    df.loc[movie,'genres'] = genres

In [228]:
df

,movieNm,movieNmEn,movieNmOg,watchGradeNm,prdtYear,showTm,genres,directors
movieCd,,,,,,,,
20179462,위대한 쇼맨,The Greatest Showman,,12세이상관람가,2017,104,"[드라마, 뮤지컬]",마이클 그레이시
20209161,언더워터,Underwater,,15세이상관람가,2019,94,[스릴러],윌리엄 유뱅크
20207893,프리즌 이스케이프,ESCAPE FROM PRETORIA,,12세이상관람가,2020,106,"[어드벤처, 스릴러]",None
20208122,카페 벨에포크,La belle epoque,,15세이상관람가,2019,115,"[코미디, 드라마]",니콜라스 베도스
20207501,그집,32 Malasana Street,Malasaña 32,15세이상관람가,2020,104,"[공포(호러), 스릴러]",None
...,...,...,...,...,...,...,...,...
20192022,마담 싸이코,Greta,,15세이상관람가,2018,98,"[공포(호러), 스릴러]",닐 조단
20196657,맨 인 블랙: 인터내셔널,Men in Black: International,,12세이상관람가,2019,115,"[액션, 코미디, SF]",F. 게리 그레이
20190885,사탄의 인형,Child's Play,,청소년관람불가,2019,90,[공포(호러)],라스 클리브버그


In [231]:
df = df.reset_index()

In [232]:
df.to_csv("movies.csv",  encoding='utf-8-sig')